In [1]:
import numpy as np
import pandas as pd

In [2]:
class Topsis:
    """ Define a TOPSIS decision making process
    TOPSIS (Technique for Order Preference by Similarity to an Ideal Solution)
    chooses and ranks alternatives of shortest distance from the ideal solution
    """
    C = None
    bad_choice = None
    bad_idx_arr = []
    count_bad = 0
    bad_idx = None
    barr_total = []
    
    def __init__(self, a, w, I):
        """ Initialise topsis object with alternatives (a), weighting (w),
        and benefit/cost indicator (i). Validate the user input for correct
        dimensions etc.

        :param np.ndarray a: A 2D array of shape (J,n)
        :param np.ndarray w: A 1D array of shape (J)
        :param np.ndarray I: A 1D array of shape (n)
        """
        # Decision Matrix
        self.a = np.array(a, dtype=np.float).T
        assert len(self.a.shape) == 2, "Decision matrix a must be 2D"

        # Number of alternatives, aspects
        (self.n, self.J) = self.a.shape

        # Weight matrix
        self.w = np.array(w, dtype=np.float)
        assert len(self.w.shape) == 1, "Weights array must be 1D"
        assert self.w.size == self.n, "Weights array wrong length, " + \
                                      "should be of length {}".format(self.n)

        # Normalise weights to 1
        self.w = self.w/sum(self.w)

        # Benefit (True) or Cost (False) criteria?
        self.I = np.array(I, dtype=np.int8)
        assert len(self.I.shape) == 1, "Criterion array must be 1D"
        assert len(self.I) == self.n, "Criterion array wrong length, " + \
                                      "should be of length {}".format(self.n)

        # Initialise best/worst alternatives lists
        ab, aw = np.zeros(self.n), np.zeros(self.n)
   
    
    def __repr__(self):
        """ What to print when the object is called?
        """
        # If good choice not yet calculated, start the calculation!
        if self.bad_choice == None:
            self.calc()

        
        
        for idx in self.bad_idx_arr:
            self.barr_total.append([idx, self.a[:, idx].tolist()])
            
        return "\n".join(('Bad alternative\n'+'a{}: {}'.format(idx[1][0], idx[1][1])) for idx in enumerate(self.barr_total))
        
    
    def step1(self):
        """ TOPSIS Step 1
        Calculate the normalised decision matrix (self.r)
        """
        self.r = self.a/np.array(np.linalg.norm(self.a, axis=1)[:, np.newaxis])
        return
    
    
    def step2(self):
        """ TOPSIS Step 2
        Calculate the weighted normalised decision matrix
        Two transposes required so that indices are multiplied correctly:
        """
        self.v = (self.w * self.r.T).T
        return
    
    
    def step3(self):
        """ TOPSIS Step 3
        Determine the ideal and negative-ideal solutions
        I[i] defines i as a member of the benefit criteria (True) or the cost
        criteria (False)
        """
        # Calcualte ideal/negative ideals
        self.ab = np.max(self.v, axis=1) * self.I + \
                  np.min(self.v, axis=1) * (1 - self.I)
        self.aw = np.max(self.v, axis=1) * (1 - self.I) +  \
                  np.min(self.v, axis=1) * self.I
        return
   
    
    def step4(self):
        """ TOPSIS Step 4
        Calculate the separation measures, n-dimensional Euclidean distance
        """
        # Create two n long arrays containing Eculidean distances
        # Save the ideal and negative-ideal solutions
        self.db = np.linalg.norm(self.v - self.ab[:,np.newaxis], axis=0)
        self.dw = np.linalg.norm(self.v - self.aw[:,np.newaxis], axis=0)
        return
    

    def step5(self):
        """ TOPSIS Step 5 & 6
        Calculate the relative closeness to the ideal solution, then rank the
        preference order
        """
        # Ignore division by zero errors
        #np.seterr(all='ignore')
        # Find relative closeness
        self.C = self.dw / (self.dw + self.db)
        get_length = len(self.C) # get eg. [1, 0.33, 0.33, 0.33]
        
        self.bad_choice = self.C.argsort()[0]
        
        print('self.C[self.bad_choice]: ',self.C[self.bad_choice])
        print('self.bad_idx_arr(before):', self.bad_idx_arr)
        for i in range(get_length):
            if self.C[i] == self.C[self.bad_choice]: #convert np.int to int  
                self.count_bad += 1
                self.bad_idx_arr.append(i)
                print('self.bad_idx_arr(in if):', self.bad_idx_arr)
        if self.count_bad == 0:
            self.bad_idx_arr.append(self.bad_choice)
            
        print('self.bad_idx_arr: ', self.bad_idx_arr) #get[1, 0, 1]???????
        return
   
    
    def calc(self):
        """ TOPSIS Calculations
        This can be called once the object is initialised, and is
        automatically called when a representation of topsis is
        needed (eg. print(topsis(matrix, weights, I)). This calls each step in
        TOPSIS algorithm and stores calcultions in self.

        The good alternatives index (starting at 0) is saved in
        self.good_choice
        """
        self.step1()
        self.step2()
        self.step3()
        self.step4()
        self.step5()
        return



In [3]:
topsis1 = Topsis([[1,3,4,5,6],[1,2,3,4,6],[1,2,3,3,6],[1,1,1,1,6],[1,1,1,1,6]]
                 , [1,1,1,1,0], [1,1,0,0,1])

In [4]:
topsis1


self.C[self.bad_choice]:  0.3522893349186756
self.bad_idx_arr(before): []
self.bad_idx_arr(in if): [1]
self.bad_idx_arr:  [1]


Bad alternative
a1: [1.0, 2.0, 3.0, 4.0, 6.0]

In [5]:
# read .csv file
df = pd.read_csv('../Desktop/vendorChoose.csv',encoding = 'big5') 

In [6]:
# Create an empty list 
Row_list =[] 
Name_list = []
my_list = []
count_col = df.shape[1] #總共幾行(包括第一行的名稱)
  
# Iterate over each row 
for index, rows in df.iterrows(): 
    # Create list for the current row 
    if my_list == []:
        for j in range(1,count_col,1):
            my_list.append(rows[j])
    else:
        my_list = []
        for j in range(1,count_col,1):
            my_list.append(rows[j])
    #my_list =[rows[1], rows[2], rows[3], rows[4]]

    # append the list to the final list 
    Row_list.append(my_list)
    Name_list.append(rows[0])

In [7]:
Row_list

[[90, 10, 100, 100, 100],
 [90, 10, 100, 100, 100],
 [80, 30, 200, 80, 100],
 [70, 30, 200, 80, 100],
 [60, 40, 200, 80, 100],
 [60, 40, 200, 80, 10]]

In [8]:
topsis2 = Topsis(Row_list, [1,1,1,1,0], [1,1,0,0,1])

In [9]:
topsis2

self.C[self.bad_choice]:  0.40099174148594535
self.bad_idx_arr(before): [1]
self.bad_idx_arr(in if): [1, 0]
self.bad_idx_arr(in if): [1, 0, 1]
self.bad_idx_arr:  [1, 0, 1]


Bad alternative
a1: [1.0, 2.0, 3.0, 4.0, 6.0]
Bad alternative
a1: [90.0, 10.0, 100.0, 100.0, 100.0]
Bad alternative
a0: [90.0, 10.0, 100.0, 100.0, 100.0]
Bad alternative
a1: [90.0, 10.0, 100.0, 100.0, 100.0]

In [10]:
topsis3 = Topsis([[1,1,1,1],[2,2,2,2],[3,3,3,3]], [1,1,1,0], [1,1,0,1])

In [11]:
topsis3

self.C[self.bad_choice]:  0.41421356237309503
self.bad_idx_arr(before): [1, 0, 1]
self.bad_idx_arr(in if): [1, 0, 1, 0]
self.bad_idx_arr:  [1, 0, 1, 0]


Bad alternative
a1: [1.0, 2.0, 3.0, 4.0, 6.0]
Bad alternative
a1: [90.0, 10.0, 100.0, 100.0, 100.0]
Bad alternative
a0: [90.0, 10.0, 100.0, 100.0, 100.0]
Bad alternative
a1: [90.0, 10.0, 100.0, 100.0, 100.0]
Bad alternative
a1: [2.0, 2.0, 2.0, 2.0]
Bad alternative
a0: [1.0, 1.0, 1.0, 1.0]
Bad alternative
a1: [2.0, 2.0, 2.0, 2.0]
Bad alternative
a0: [1.0, 1.0, 1.0, 1.0]